In [ ]:
from google.colab import files
files.upload()  # faça upload do kaggle.json aqui

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d leaderandpiller/brain-tumor-segmentation


Dataset URL: https://www.kaggle.com/datasets/leaderandpiller/brain-tumor-segmentation
License(s): unknown
 96% 537M/560M [00:01<00:00, 323MB/s]
100% 560M/560M [00:01<00:00, 417MB/s]


In [ ]:
import zipfile

with zipfile.ZipFile("brain-tumor-segmentation.zip", 'r') as zip_ref:
    zip_ref.extractall("brain_tumor_data")


In [ ]:
import os

for root, dirs, files in os.walk("brain_tumor_data"):
    print(f"{root} -> {len(files)} arquivos, subpastas: {dirs}")


brain_tumor_data -> 0 arquivos, subpastas: ['YES-20200502T082251Z-001', 'NO-20200502T075011Z-001']
brain_tumor_data/YES-20200502T082251Z-001 -> 0 arquivos, subpastas: ['YES']
brain_tumor_data/YES-20200502T082251Z-001/YES -> 5504 arquivos, subpastas: []
brain_tumor_data/NO-20200502T075011Z-001 -> 0 arquivos, subpastas: ['NO']
brain_tumor_data/NO-20200502T075011Z-001/NO -> 6159 arquivos, subpastas: []


In [ ]:
import os
import shutil

base_path = "brain_tumor_data"
combined_path = os.path.join(base_path, "combined")

os.makedirs(combined_path, exist_ok=True)
os.makedirs(os.path.join(combined_path, "YES"), exist_ok=True)
os.makedirs(os.path.join(combined_path, "NO"), exist_ok=True)

# mover imagens YES
src_yes = os.path.join(base_path, "YES-20200502T082251Z-001", "YES")
for f in os.listdir(src_yes):
    shutil.copy(os.path.join(src_yes, f), os.path.join(combined_path, "YES", f))

# mover imagens NO
src_no = os.path.join(base_path, "NO-20200502T075011Z-001", "NO")
for f in os.listdir(src_no):
    shutil.copy(os.path.join(src_no, f), os.path.join(combined_path, "NO", f))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

"""
A Compose serve para aplicar a pipeline, fazendo com que possamos aplicar
vários processamentos de uma vez só nas imagens ou conjunto de imagens
"""
transformacao = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

dataset = datasets.ImageFolder(
    root='brain_tumor_data/combined',
    transform=transformacao
)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

class Classificador(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3))
        self.conv2 = nn.Conv2d(32, 32, (3, 3))
        self.activation = nn.ReLU()
        self.bnorm = nn.BatchNorm2d(num_features=32)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(in_features=32 * 30 * 30, out_features=128)
        self.linear2 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 1)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, X):
        X = self.pool(self.bnorm(self.activation(self.conv1(X))))
        X = self.pool(self.bnorm(self.activation(self.conv2(X))))
        X = self.flatten(X)
        X = self.dropout(self.activation(self.linear1(X)))
        X = self.dropout(self.activation(self.linear2(X)))
        X = self.output(X)
        return X

classificador = Classificador()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classificador.parameters(), lr=1e-4)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
classificador = classificador.to(device)

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
num_epocas = 4
for epoch in range(num_epocas):
    classificador.train()
    running_loss = 0.0
    running_correct = 0
    total = 0

    for imgs, labels in tqdm(train_loader, desc=f"🌱 Treinando Época {epoch+1}"):
        imgs, labels = imgs.to(device), labels.to(device).float().view(-1, 1)

        optimizer.zero_grad()
        outputs = classificador(imgs).squeeze(1)
        loss = criterion(outputs, labels.squeeze(1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)

        with torch.no_grad():
            preds = torch.sigmoid(outputs)
            preds = (preds > 0.5).float()
            running_correct += (preds == labels.squeeze(1)).sum().item()
            total += labels.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = running_correct / total
    print(f"✅ Época {epoch+1} - Loss Treino: {train_loss:.4f} | Acurácia Treino: {train_acc:.4f}")

    classificador.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device).float().view(-1, 1)

            outputs = classificador(imgs).squeeze(1)
            loss = criterion(outputs, labels.squeeze(1))
            val_loss += loss.item() * imgs.size(0)

            preds = torch.sigmoid(outputs)
            preds = (preds > 0.5).float()
            val_correct += (preds == labels.squeeze(1)).sum().item()
            val_total += labels.size(0)

    val_epoch_loss = val_loss / len(val_loader.dataset)
    val_accuracy = val_correct / val_total
    print(f"🔍 Validação — Loss: {val_epoch_loss:.4f} | Acurácia: {val_accuracy:.4f}")

    train_losses.append(train_loss)
    val_losses.append(val_epoch_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_accuracy)



🌱 Treinando Época 1: 100%|██████████| 292/292 [00:29<00:00,  9.82it/s]


✅ Época 1 - Loss Treino: 0.1550 | Acurácia Treino: 0.9388
🔍 Validação — Loss: 0.0672 | Acurácia: 0.9786


🌱 Treinando Época 2: 100%|██████████| 292/292 [00:29<00:00,  9.98it/s]


✅ Época 2 - Loss Treino: 0.0127 | Acurácia Treino: 0.9968
🔍 Validação — Loss: 0.0485 | Acurácia: 0.9811


🌱 Treinando Época 3: 100%|██████████| 292/292 [00:28<00:00, 10.08it/s]


✅ Época 3 - Loss Treino: 0.0024 | Acurácia Treino: 0.9998
🔍 Validação — Loss: 0.0782 | Acurácia: 0.9623


🌱 Treinando Época 4: 100%|██████████| 292/292 [00:28<00:00, 10.10it/s]


✅ Época 4 - Loss Treino: 0.0008 | Acurácia Treino: 1.0000
🔍 Validação — Loss: 0.0162 | Acurácia: 0.9961


In [ ]:
class TrainTumorClass(nn.Module):
  def __init__(self):
    super.__init__(self, TrainTumorClass)
    self.conv11 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), padding=1)
    self.conv2 = nn.Conv2d(32,64,(3,3),padding=1)
    self.relu = nn.ReLU()
    self.bnorm = nn.BatchNorm2d(64)
    self.pool = nn.MaxPool2d(kernel_size=(2,2))
    self.flatten = nn.Flatten()